# Configuration 

In [124]:
import os
from openai import OpenAI



from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()



AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = "gpt-4o-mini"  

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT


# Create client for Azure OpenAI
client = OpenAI(
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    base_url=f"{os.environ['AZURE_OPENAI_ENDPOINT']}/openai/v1/",
)

# Simple test call
resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT, 
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Introduce your self in short sentence."},
    ],
)

print(resp.choices[0].message.content)


I am an AI language model designed to assist with information, answer questions, and facilitate engaging conversations.


# Check the language syntax 

In [125]:
def normalize_query(text: str) -> str:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": "You understand poorly written Norwegian and English. Correct spelling mistakes, interpret the meaning, and rewrite the sentence clearly."},
            {"role": "user", "content": text},
        ],
        temperature=0.1
    )
    return resp.choices[0].message.content.strip()


# LLM planner: convert the natural language query to a structured JSON plan


In [193]:
import json

SYSTEM_PROMPT = """


You are a GIS planner agent for a Nordic municipality.
You NEVER execute SQL and NEVER touch the database.
You ONLY output a JSON plan that another system will translate to SQL.

------------------------------------------------------------
OUTPUT FORMAT (STRICT)
------------------------------------------------------------
You MUST output ONLY valid JSON with EXACTLY these fields:

{
  "operation": "...",
  "layer": "...",
  "target_layer": "...",
  "buffer_meters": ...,
  "limit": ...,
  "where_clause": "..."
}




Rules:
- No backticks, no explanations, no comments.
- All fields MUST exist, even if null or empty.
- buffer_meters MUST be a number or null.
- limit MUST be a number or null.
- where_clause MUST be either "" or a simple phrase (see rules below).
- layer and target_layer MUST be valid or "".


------------------------------------------------------------
ALLOWED OPERATIONS
------------------------------------------------------------
General:
- "select_limit_only"
- "select_by_attribute"
- "select_buffer"
- "select_intersect"
- "select_nearest"
- "select_within_polygon"

Special (only if user explicitly asks):
- Buildings: "select_buildings_in_floodzone", "select_buildings_near_route", "select_buildings_by_area"
- Flood zones: "select_within_floodzone", "select_intersect_floodzone"
- Bicycle routes: "select_near_bikeroute", "select_intersect_bikeroute"
- Walking routes: "select_near_walkroute", "select_intersect_walkroute"
- Ski routes: "select_near_skiroute", "select_intersect_skiroute"
- Route info points: "select_nearest_rutepoint", "select_points_in_area"



------------------------------------------------------------
ALLOWED LAYERS (MUST MATCH EXACTLY)
------------------------------------------------------------
- "buildings"
- "flomsoner"
- "buildings_sample"
- "arealbruk_skogbonitet_sample"
- "flomsoner_sample"
- "sykkelrute_senterlinje_sample"
- "skiloype_senterlinje"
- "annenrute_senterlinje"
- "annenruteinfo_tabell"
- "arealbruk_skogbonitet"
- "fotrute_senterlinje"
- "fotruteinfo_tabell"
- "ruteinfopunkt_posisjon"
- "skiloypeinfo_tabell"
- "sykkelrute_senterlinje"
- "sykkelruteinfo_tabell"

If no layer is clearly referenced → layer = "".






------------------------------------------------------------
target_layer RULES (VERY IMPORTANT)
------------------------------------------------------------
For ANY spatial operation (buffer, intersect, nearest),
the JSON MUST include a valid "target_layer".

Mapping:
- "near water" → "flomsoner"
- "near river" → "flomsoner"
- "intersect floodzone" → "flomsoner"
- "inside floodzone" → "flomsoner"
- "near bikeroute" → "sykkelrute_senterlinje"
- "intersect bikeroute" → "sykkelrute_senterlinje"
- "near walkroute" → "fotrute_senterlinje"
- "near skiroute" → "skiloype_senterlinje"

If user gives NO spatial relation → target_layer MUST be "".



------------------------------------------------------------
OPERATION SELECTION RULES
------------------------------------------------------------
- If the user says “within X meters” → operation = "select_buffer".
- If the user says “near X” → operation = "select_buffer", unless they say “nearest”.
- If the user says “nearest” or “closest” → operation = "select_nearest".
- If no spatial relation is described → DO NOT choose buffer/intersect/nearest.
- City names (e.g., “Kristiansand”) MUST NOT produce filters.

------------------------------------------------------------
LIMIT RULES
------------------------------------------------------------
- If user gives a number (e.g., “10 buildings”) → use it.
- If user does not specify → limit = null.
- If the user writes exactly "all" → limit = "all".
- If the user says "all buildings", "all houses", etc. → limit = "all".

------------------------------------------------------------
BUFFER RULES
------------------------------------------------------------
- If user says “within X meters”, set buffer_meters = X.
- Otherwise buffer_meters = null.

------------------------------------------------------------
WHERE_CLAUSE RULES
------------------------------------------------------------
Only fill where_clause if the user explicitly says:
- near water
- near river
- near bikeroute
- inside floodzone
- intersect floodzone
- etc.

Otherwise: where_clause = "".

------------------------------------------------------------
FINAL INSTRUCTION
------------------------------------------------------------
Your entire output MUST be a single JSON object following all rules.
No prose. No Markdown. No explanations. Only JSON.

"""


previous_input = None

def plan_spatial_query(nl_query: str) -> dict:
    resp = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": nl_query},
        ],
        temperature=0.0,
        max_tokens=300,
    )
    raw = resp.choices[0].message.content.strip()
    if raw.startswith("```"):
        raw = raw.strip("`")
        if raw.lower().startswith("json"):
            raw = raw[4:].strip()
    return json.loads(raw)



# Process user input

In [208]:
FIELD_INFO = {
    # ****************************************************
    "operation": (
        "Type of GIS action.\n"
        "Examples:\n"
        "- select_buffer\n"
        "- select_intersect\n"
        "- select_nearest\n"
        "- select_limit_only\n"
        "- select_by_attribute\n"
        "This tells the GIS engine WHAT to do."
    ),

    # ****************************************************
    "layer": (
        "The PRIMARY dataset you want results from.\n"
        "Must match one of the allowed database layers:\n"
        "- buildings\n"
        "- flomsoner\n"
        "- buildings_sample\n"
        "- arealbruk_skogbonitet_sample\n"
        "- flomsoner_sample\n"
        "- sykkelrute_senterlinje_sample\n"
        "- skiloype_senterlinje\n"
        "- annenrute_senterlinje\n"
        "- annenruteinfo_tabell\n"
        "- arealbruk_skogbonitet\n"
        "- fotrute_senterlinje\n"
        "- fotruteinfo_tabell\n"
        "- ruteinfopunkt_posisjon\n"
        "- skiloypeinfo_tabell\n"
        "- sykkelrute_senterlinje\n"
        "- sykkelruteinfo_tabell"
    ),

    # ****************************************************
    "target_layer": (
        "The SECOND dataset used for spatial relation.\n"
        "ONLY required for operations involving two layers (buffer, intersect, nearest).\n"
        "Examples:\n"
        "- layer = buildings\n"
        "- target_layer = sykkelrute_senterlinje  -> buildings near bike routes\n"
        "- target_layer = flomsoner               -> buildings intersect floodzones\n"
        "\nIf operation does not require a second dataset → target_layer must be null."
    ),

    # ****************************************************
    "buffer_meters": (
        "Distance in meters for spatial proximity.\n"
        "Examples:\n"
        "- 50\n"
        "- 100\n"
        "- 200\n"
        "Used only for operations requiring distance (select_buffer, select_near_...)."
    ),

    # ****************************************************
    "limit": (
        "How many results to return.\n"
        "Examples: 5, 10, 50.\n"
            ),
    


    # ****************************************************
    "where_clause": (
        "Optional simple spatial or attribute filter.\n"
        "Only filled when the user explicitly mentions a real filter:\n"
        "- near river\n"
        "- inside floodzone\n"
        "- intersect bikeroute\n"
        "- type = 'house'\n"
        "\nIf user does NOT specify a filter → where_clause = \"\"."
    )
}



def process_user_input(user_input):
    
    clean_text = normalize_query(user_input)

    plan = plan_spatial_query(clean_text)


    lower_input = user_input.lower()
    
    if "all " in lower_input:
        plan["limit"] = "all"
   
    required_fields = ["operation", "layer", "buffer_meters", "limit", "target_layer"]
    missing = []




    
    limit = plan.get("limit")

    if isinstance(limit, str) and limit.lower() == "all":
        plan["limit"] = 100
    else:
        plan["limit"] = limit  

    # Check missing fields
    for f in required_fields:
        if plan.get(f) is None or plan.get(f) == "":
            missing.append(f)

    # If missing → return helpful message
    if missing:
        explanations = "\n".join([f"- {f}: {FIELD_INFO[f]}" for f in missing])

        return (
            f"\n⚠ Missing required field(s): {', '.join(missing)}\n"
            + "\n----------------------------------------\n"
            + f"{explanations}\n"
            + "----------------------------------------\n"
            + f"Your input:\n  {user_input}\n"
            + "----------------------------------------\n"
        )

        

    return plan


# Test the LLM planner

In [195]:

query = "Find 100 residential houses within 200 meters of a river in Kristiansand."
plan = plan_spatial_query(query)
plan


{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': ''}

In [196]:

query = "Find all buildings within 100 m of bicycle routes"
plan = plan_spatial_query(query)
plan


{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'sykkelrute_senterlinje',
 'buffer_meters': 100,
 'limit': None,
 'where_clause': ''}

# Converts the user’s prompt into an SQL query string.

In [197]:
def plan_to_sql(plan: dict) -> str:
    op = plan["operation"]
    layer = plan["layer"]
    target = plan.get("target_layer")  
    buf = plan.get("buffer_meters")
    limit = plan.get("limit") or 100
    where = plan.get("where_clause") or "TRUE"

    # SELECT LIMIT ONLY
    if op == "select_limit_only":
        return f"""
SELECT *
FROM public.{layer}
LIMIT {limit};
""".strip()

    # SELECT BY ATTRIBUTE
    if op == "select_by_attribute":
        return f"""
SELECT *
FROM public.{layer}
WHERE {where}
LIMIT {limit};
""".strip()

    # BUFFER OPERATION (A near B)
    if op == "select_buffer":
        if not target:
            raise ValueError("select_buffer requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
JOIN (
    SELECT ST_Buffer(geom, {buf}) AS geom
    FROM public.{target}
) t
ON ST_Intersects(a.geom, t.geom)
WHERE {where}
LIMIT {limit};
""".strip()

    # INTERSECT OPERATION (A intersects B)
    if op == "select_intersect":
        if not target:
            raise ValueError("select_intersect requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
JOIN public.{target} b
  ON ST_Intersects(a.geom, b.geom)
WHERE {where}
LIMIT {limit};
""".strip()

    # NEAREST OPERATION (A nearest to B)
    if op == "select_nearest":
        if not target:
            raise ValueError("select_nearest requires 'target_layer'")
        return f"""
SELECT a.*
FROM public.{layer} a
ORDER BY (
    SELECT MIN(ST_Distance(a.geom, b.geom))
    FROM public.{target} b
)
LIMIT {limit};
""".strip()

    # WITHIN POLYGON
    if op == "select_within_polygon":
        polygon = plan["where_clause"]
        return f"""
SELECT *
FROM public.{layer}
WHERE ST_Within(geom, ST_GeomFromText('{polygon}', 4326))
LIMIT {limit};
""".strip()

    raise ValueError(f"Unsupported operation: {op}")


# Connect to the PostGIS database and run the SQL query, returning a DataFrame


In [198]:
import psycopg2
import pandas as pd

def run_postgis_query(sql: str) -> pd.DataFrame:
    conn_str = os.environ["PGCONN_STRING"]
    
    
    with psycopg2.connect(conn_str) as conn:
        
        with conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=cols)


# Execute the GIS plan in PostGIS and return the result as a DataFrame


In [199]:
def execute_gis_plan_db(plan: dict):
    sql = plan_to_sql(plan)
    df = run_postgis_query(sql)
    return df

# Ask the GIS agent: NL query -> LLM plan -> SQL -> PostGIS -> DataFrame


In [200]:
def ask_gis_agent(query: str) -> pd.DataFrame:



    
    
    plan = process_user_input(query)

    # If the agent returned a string → it's an error message
    if isinstance(plan, str):
        print(plan)
        return None  # STOP HERE

    
    df = execute_gis_plan_db(plan)
    return df

# Send the query to the GIS agent and display the first results


In [182]:

queryy = "Find 100 residential houses within 200 meters of a river in Kristiansand."
plan = plan_spatial_query(query)
plan


{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': ''}

In [183]:

queryy = "Find all buildings within 100 m of bicycle routes."
plan = plan_spatial_query(query)
plan


{'operation': 'select_buffer',
 'layer': 'buildings',
 'target_layer': 'flomsoner',
 'buffer_meters': 200,
 'limit': 100,
 'where_clause': ''}

In [201]:
query = "Find 100 residential houses within 200 meters of a river in Kristiansand."
df = ask_gis_agent(query)
df


,gid,osm_id,code,fclass,name,type,geom
0,2623657,954371361,1500,building,None,house,0106000020E96400000100000001030000000100000005...
1,2623658,954371362,1500,building,None,garage,0106000020E96400000100000001030000000100000005...
2,2623706,954371410,1500,building,None,garage,0106000020E96400000100000001030000000100000007...
3,2623763,954371467,1500,building,None,house,0106000020E96400000100000001030000000100000005...
4,2623635,954371339,1500,building,None,house,0106000020E9640000010000000103000000010000000B...
...,...,...,...,...,...,...,...
95,2624955,954372659,1500,building,None,house,0106000020E96400000100000001030000000100000007...
96,2624958,954372662,1500,building,None,house,0106000020E96400000100000001030000000100000009...
97,2624959,954372663,1500,building,None,warehouse,0106000020E96400000100000001030000000100000005...
98,2624962,954372666,1500,building,None,garage,0106000020E96400000100000001030000000100000005...


In [209]:
 res = ask_gis_agent("c")
res


⚠ Missing required field(s): operation, layer, buffer_meters, limit, target_layer

----------------------------------------
- operation: Type of GIS action.
Examples:
- select_buffer
- select_intersect
- select_nearest
- select_limit_only
- select_by_attribute
This tells the GIS engine WHAT to do.
- layer: The PRIMARY dataset you want results from.
Must match one of the allowed database layers:
- buildings
- flomsoner
- buildings_sample
- arealbruk_skogbonitet_sample
- flomsoner_sample
- sykkelrute_senterlinje_sample
- skiloype_senterlinje
- annenrute_senterlinje
- annenruteinfo_tabell
- arealbruk_skogbonitet
- fotrute_senterlinje
- fotruteinfo_tabell
- ruteinfopunkt_posisjon
- skiloypeinfo_tabell
- sykkelrute_senterlinje
- sykkelruteinfo_tabell
- buffer_meters: Distance in meters for spatial proximity.
Examples:
- 50
- 100
- 200
Used only for operations requiring distance (select_buffer, select_near_...).
- limit: How many results to return.
Examples: 5, 10, 50.

- target_layer: The 

# Simple CLI chat loop that sends user queries to the GIS agent and prints the results


In [206]:
def chat_loop():
    # previous_input = None
    print("GIS agent chat – type 'quit' to stop.\n")

    while True:
        user_q = input("You: ").strip()
        
        print("'''''''''''''''''''''''''''''''''''''''''''")
    


        if user_q.lower() in ("quit", "exit", "q"):
            print("Welcome back")
            break


        # if user_q in ["new", "restart", "rest"]:
        #     previous_input = None
        #     continue

        # # If we had missing fields earlier → append the new text
        # if previous_input is not None:
        #     combined = previous_input + " " + user_q
        #     user_q = combined

        result = ask_gis_agent(user_q)

       # If result is an error string → print it
        if isinstance(result, str):
            print(result)
            continue
        
        # Otherwise print DataFrame

        display(result)

        # previous_input = None  # reset when success
chat_loop()

GIS agent chat – type 'quit' to stop.



You:  Find all buildings within 100 m of bicycle routes.


'''''''''''''''''''''''''''''''''''''''''''


,gid,osm_id,code,fclass,name,type,geom
0,2624367,954372071,1500,building,None,house,0106000020E96400000100000001030000000100000009...
1,2624369,954372073,1500,building,None,garage,0106000020E96400000100000001030000000100000005...
2,2624370,954372074,1500,building,None,garage,0106000020E96400000100000001030000000100000005...
3,2624374,954372078,1500,building,None,house,0106000020E96400000100000001030000000100000013...
4,2624460,954372164,1500,building,None,garage,0106000020E96400000100000001030000000100000005...
...,...,...,...,...,...,...,...
95,3474309,1004848000,1500,building,None,shed,0106000020E96400000100000001030000000100000005...
96,3474310,1004848001,1500,building,None,cabin,0106000020E96400000100000001030000000100000007...
97,3465500,1004839178,1500,building,None,None,0106000020E96400000100000001030000000100000005...
98,3465501,1004839179,1500,building,None,cabin,0106000020E96400000100000001030000000100000005...


You:  q


'''''''''''''''''''''''''''''''''''''''''''
Welcome back


In [192]:
def chat_loop():
    print("GIS agent chat – type 'quit' to stop.\n")

    while True:
        user_q = input("You: ").strip()
        
        print("'''''''''''''''''''''''''''''''''''''''''''")

        if user_q.lower() in ("quit", "exit", "q"):
            print("Welcome back")
            break

        result = ask_gis_agent(user_q)

        # Missing fields → now result contains the error message (string)
        if isinstance(result, str):
            print(result)
            continue

        print(result)
chat_loop()

GIS agent chat – type 'quit' to stop.



You:  Find all buildings within 100 m of bicycle routes


'''''''''''''''''''''''''''''''''''''''''''
        gid      osm_id  code    fclass              name    type  \
0   2624367   954372071  1500  building              None   house   
1   2624369   954372073  1500  building              None  garage   
2   2624370   954372074  1500  building              None  garage   
3   2624374   954372078  1500  building              None   house   
4   2624460   954372164  1500  building              None  garage   
..      ...         ...   ...       ...               ...     ...   
95  3474309  1004848000  1500  building              None    shed   
96  3474310  1004848001  1500  building              None   cabin   
97  3465500  1004839178  1500  building              None    None   
98  3465501  1004839179  1500  building              None   cabin   
99   214919   187639370  1500  building  Deanu Sàmeskuvla  school   

                                                 geom  
0   0106000020E96400000100000001030000000100000009...  
1   0106000020E

You:  q


'''''''''''''''''''''''''''''''''''''''''''
Welcome back


In [168]:
import psycopg2
import pandas as pd

def run_postgis_query(sql: str) -> pd.DataFrame:
    conn_str = os.environ["PGCONN_STRING"]
    
    
    with psycopg2.connect(conn_str) as conn:
        
        with conn.cursor() as cur:
            cur.execute(sql)
            rows = cur.fetchall()
            cols = [desc[0] for desc in cur.description]
    return pd.DataFrame(rows, columns=cols)


In [169]:
run_postgis_query("""
SELECT table_schema, table_name 
FROM information_schema.tables 
WHERE table_schema='public';
""")


,table_schema,table_name
0,public,buildings
1,public,flomsoner
2,public,buildings_sample
3,public,arealbruk_skogbonitet_sample
4,public,flomsoner_sample
5,public,sykkelrute_senterlinje_sample
6,public,skiloype_senterlinje
7,public,annenrute_senterlinje
8,public,annenruteinfo_tabell
9,public,arealbruk_skogbonitet


In [170]:
run_postgis_query("""
SELECT column_name, data_type 
FROM information_schema.columns 
WHERE table_name='buildings';
""")


,column_name,data_type
0,code,integer
1,gid,integer
2,geom,USER-DEFINED
3,type,character varying
4,fclass,character varying
5,osm_id,character varying
6,name,character varying


In [171]:
run_postgis_query("SELECT * FROM buildings LIMIT 50;")


,gid,osm_id,code,fclass,name,type,geom
0,506461,508534831,1500,building,None,apartments,0106000020E96400000100000001030000000100000005...
1,506462,508534832,1500,building,None,apartments,0106000020E96400000100000001030000000100000027...
2,506463,508534833,1500,building,None,apartments,0106000020E9640000010000000103000000010000001B...
3,506464,508534834,1500,building,None,detached,0106000020E96400000100000001030000000100000009...
4,506465,508534835,1500,building,None,retail,0106000020E96400000100000001030000000100000007...
5,506466,508534836,1500,building,Salmakergården,apartments,0106000020E96400000100000001030000000100000017...
6,506467,508534837,1500,building,None,None,0106000020E96400000100000001030000000100000009...
7,506468,508534840,1500,building,None,detached,0106000020E96400000100000001030000000100000009...
8,506469,508534841,1500,building,None,detached,0106000020E96400000100000001030000000100000005...
9,506470,508534842,1500,building,None,detached,0106000020E96400000100000001030000000100000009...


In [174]:
run_postgis_query("""
SELECT b.*
FROM public.buildings b
JOIN public.sykkelrute_senterlinje r
  ON ST_DWithin(b.geom, r.geom, 100)
LIMIT 50;
""")


,gid,osm_id,code,fclass,name,type,geom
0,506461,508534831,1500,building,None,apartments,0106000020E96400000100000001030000000100000005...
1,506462,508534832,1500,building,None,apartments,0106000020E96400000100000001030000000100000027...
2,506462,508534832,1500,building,None,apartments,0106000020E96400000100000001030000000100000027...
3,506463,508534833,1500,building,None,apartments,0106000020E9640000010000000103000000010000001B...
4,506463,508534833,1500,building,None,apartments,0106000020E9640000010000000103000000010000001B...
5,506464,508534834,1500,building,None,detached,0106000020E96400000100000001030000000100000009...
6,506465,508534835,1500,building,None,retail,0106000020E96400000100000001030000000100000007...
7,506466,508534836,1500,building,Salmakergården,apartments,0106000020E96400000100000001030000000100000017...
8,506467,508534837,1500,building,None,None,0106000020E96400000100000001030000000100000009...
9,506467,508534837,1500,building,None,None,0106000020E96400000100000001030000000100000009...


In [175]:
run_postgis_query("SELECT * FROM sykkelrute_senterlinje LIMIT 50;")


,gid,objtype,skilting,anleggsnummer,uukoblingsid,belysning,lokalid,navnerom,versjonid,datafangstdato,...,informasjon,merking,rutefolger,underlagstype,rutebredde,trafikkbelastning,sesong,malemetode,shape_length,geom
0,1,Sykkelrute,,None,None,,02989546-4938-49c9-9405-906bafc7c54a,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1971-09-09 00:00:00+00:00,...,Generert fra Traktorvegkant,JA,TR,None,None,None,,60,523.414390,0105000020E9640000010000000102000000DF00000000...
1,2,Sykkelrute,,None,None,,090afebf-7920-4765-965e-c16024d70146,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1998-06-08 00:00:00+00:00,...,None,JA,BV,None,None,None,,22,3194.542971,0105000020E964000001000000010200000079000000D0...
2,3,Sykkelrute,,None,None,,0ea52391-5f2e-4bc1-aacd-2f7a6aeb0b38,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2009-11-18 00:00:00+00:00,...,Kombinert med sykkelrute,JA,GS,None,None,None,,60,32.429166,0105000020E96400000100000001020000000700000040...
3,4,Sykkelrute,,None,None,,386f947d-5bc1-45f0-8f5b-0a6124459861,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2014-09-17 00:00:00+00:00,...,None,JA,ST,None,None,None,,24,171.768400,0105000020E96400000100000001020000001F00000000...
4,5,Sykkelrute,,None,None,,253e48ab-d5e7-42b9-83f9-3a48c531e6cd,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2012-08-10 00:00:00+00:00,...,None,JA,TR,None,None,None,,24,35.215468,0105000020E96400000100000001020000000700000090...
5,6,Sykkelrute,,None,None,,3132db05-8e9a-4e53-a5fd-fb87df5d337e,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1996-06-13 00:00:00+00:00,...,None,JA,BV,None,None,None,,22,384.713777,0105000020E964000001000000010200000018000000E8...
6,7,Sykkelrute,,None,None,,12f58116-95e9-404b-ae98-4715e978914c,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,1997-07-01 00:00:00+00:00,...,None,JA,ST,None,None,None,,22,3436.603532,0105000020E96400000100000001020000007C00000010...
7,8,Sykkelrute,,None,None,,157295ca-bd66-4fb3-b143-c7ec714b068b,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2016-08-09 00:00:00+00:00,...,None,JA,ST,None,None,None,,92,1263.105798,0105000020E964000001000000010200000076000000A8...
8,9,Sykkelrute,,None,None,,19bea04d-34f8-4f49-b4b7-67c5e6884357,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2008-08-19 00:00:00+00:00,...,None,JA,TR,None,None,None,,24,1255.101054,0105000020E96400000100000001020000008300000010...
9,10,Sykkelrute,,None,None,,97bd7a36-8f28-4f5e-9f4c-b39aeee3f41f,http://data.geonorge.no/TurruterNGIS/Turruter/so,2023-10-25 11:30:57.527772000,2014-09-17 00:00:00+00:00,...,None,JA,ST,None,None,None,,24,146.498739,0105000020E96400000100000001020000001300000010...


In [176]:

run_postgis_query("SELECT * FROM flomsoner LIMIT 50;")


,gid,objid,objtype,lavpunkt,gjentaksintervall,forstedigitaliseringsdato,noyaktighet,noyaktighethoyde,statusdato,flomsoneid,...,versjonid,datauttaksdato,opphav,symbolflom,malemetode,malemetodehoyde,statuskartlegging,shape_length,shape_area,geom
0,1,1,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,35.001523,87.041427,0106000020E9640000010000000103000000010000001D...
1,2,2,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,83.620072,177.187296,0106000020E96400000100000001030000000100000038...
2,3,3,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,50.070632,105.886299,0106000020E96400000100000001030000000100000032...
3,4,4,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,71.986928,249.904775,0106000020E96400000100000001030000000100000023...
4,5,5,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,150.904263,523.842873,0106000020E96400000100000001030000000100000078...
5,6,6,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,2206.783609,27629.307678,0106000020E96400000100000001030000000100000095...
6,7,7,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,42.220974,108.129053,0106000020E96400000100000001030000000100000013...
7,8,8,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,1784.441617,46832.829341,0106000020E96400000100000001030000000300000033...
8,9,9,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,14687.411504,936949.009108,0106000020E9640000010000000103000000100000006D...
9,10,10,FlomAreal,0,500,2002-03-22 00:00:00+00:00,36,None,2002-03-22 00:00:00+00:00,fs234_4,...,1.1,2025-03-16 09:00:34+00:00,NVE,1,61,None,1,5360.759072,430969.374874,0106000020E96400000100000001030000000800000080...
